In [1]:
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
matplotlib.use("agg")

from exp_spec_info import *
from plot_info import *
from select_data import *

In [2]:
# Processed pickle paths
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Summary output data
csv_output_dir = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\csvsimple-data"
summary_plot_dir = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\body_chapters\\chap_6\\images"
os.makedirs(summary_plot_dir, exist_ok=True)

# Plot output directory
chap_3_plot_dir = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\body_chapters\\chap_3\\images"
os.makedirs(chap_3_plot_dir, exist_ok=True)

In [3]:
# Load data
extended_data = pd.read_pickle(extended_data_path)
median_data = pd.read_pickle(median_data_path)

##### 3 Generate Ideal and Non-Ideal Example Cases

In [4]:
experiments_spec_to_plot = [
    ("FEM_3D_thermal1", "unprecond", 30),
    ("marine1", "unprecond", 30)
]

for matrix, setup, restart_param in experiments_spec_to_plot:

    sub_data = df_sel_setup_matrix_restart(
        extended_data, setup, matrix, restart_param
    )

    fig, ax = plt.subplots(figsize=(6.4, 3.6))

    for fp_solver in FP_SOLVERS:
        plot_data = sub_data[sub_data["solver"] == fp_solver]
        plot_exp_iters_conv_traj(
            ax,
            plot_data,
            N_EXPERIMENT_ITERATIONS,
            fp_solver,
            SOLVER_CLR_DICT[fp_solver]     
        )

    ax.legend()
    ax.grid()
    ax.set_ylabel("$\\|b-Ax_{i}\\|_{2}/\\|b-Ax_{0}\\|_{2}$")
    ax.set_xlabel("Inner Iteration")
    
    fig.suptitle(
        f"{matrix} {SETUP_NAME_MAPPING[setup]} GMRES({restart_param}) Convergence"
    )
    
    plt.savefig(
        os.path.join(chap_3_plot_dir, f"{setup}_{matrix}_{restart_param}.png"),
        bbox_inches="tight"
    )

    plt.close()
    

##### 6 Summary Scatter

In [5]:
scale = 1.4
restart_param_scale_map = {
    10: 10*scale,
    20: 20*scale,
    30: 30*scale,
    40: 40*scale,
    50: 50*scale,
    100: 60*scale,
    150: 70*scale,
    200: 80*scale,
}
restart_param_alpha_map = {
    10: 1.,
    20: 0.95,
    30: 0.9,
    40: 0.85,
    50: 0.8,
    100: 0.75,
    150: 0.7,
    200: 0.65,
}

m_rows = 3
n_cols = 2
for setup in SETUPS[:1]:

    iter = 0
    plot_iter = 0
    total_plots = len(SETUP_MATRIX_MAPPING[setup])

    for matrix in SETUP_MATRIX_MAPPING[setup]:

        if (iter % (m_rows*n_cols)) == 0:
            fig, axs = plt.subplots(m_rows, n_cols, figsize=(n_cols*3.5, m_rows*3.5))
            axs_flat = axs.flatten()
        ax = axs_flat[(iter % (m_rows*n_cols))]

        sub_data = df_sel_setup_matrix(median_data, setup, matrix)

        for solver in GMRES_M_SOLVERS:
            for restart_param in RESTART_PARAMS:
                plot_data = sub_data[
                    (sub_data["solver"] == solver) &
                    (sub_data["restart_param"] == restart_param)
                ]
                ax.scatter(
                    plot_data["med_rel_time"],
                    plot_data["med_rel_res_frac_err"],
                    plot_data["restart_param"].apply(
                        lambda x: restart_param_scale_map[x]
                    ),
                    alpha=restart_param_alpha_map[restart_param],
                    color=SOLVER_CLR_DICT[solver],
                    zorder=2
                )
        ax.axhline(0., linestyle=":", color=SOLVER_CLR_DICT["FP FP64"], zorder=1)
        ax.axvline(1.0, linestyle=":", color=SOLVER_CLR_DICT["FP FP64"], zorder=1)

        ax.set_title(f"{matrix}")
        ax.grid()
        ax.set_axisbelow(True)

        if (((iter+1) % (m_rows*n_cols)) == 0) or ((iter+1) == total_plots):
            for ax in axs[:, 0]:
                ax.set_ylabel("Median Rel. Res. Frac. Error")
            n_to_remove = ((m_rows*n_cols) - ((iter+1) % (m_rows*n_cols))) % (m_rows*n_cols)
            for i in range(n_to_remove):
                axs_flat[(m_rows*n_cols-1)-i].set_axis_off()
            last_row_ind = int(np.ceil((m_rows*n_cols-n_to_remove)/n_cols))-1
            n_in_last_row = n_to_remove % n_cols
            if n_in_last_row > 0:
                for i in range(n_in_last_row):
                    axs[last_row_ind, i].set_xlabel("Median Relative Time")
                for i in range(n_cols-n_in_last_row):
                    axs[last_row_ind-1, n_cols-1-i].set_xlabel("Median Relative Time")
            else:
                for ax in axs[last_row_ind, :]:
                    ax.set_xlabel("Median Relative Time")

            fig.tight_layout()
            fig.savefig(
                os.path.join(summary_plot_dir, f"summary-{plot_iter}.png")
            )

            plot_iter += 1
        
        iter += 1